## Vision Transformer

In [ ]:
# Imports

import tensorflow as tf
from transformers import ViTFeatureExtractor, TFViTForImageClassification

tf.keras.backend.set_image_data_format('channels_first')

In [ ]:
model_path = 'google/vit-base-patch16-224'
model = TFViTForImageClassification.from_pretrained(model_path)
feature_extractor = ViTFeatureExtractor.from_pretrained(model_path)
feature_extractor

In [ ]:
for image, label in temp_set.take(1):
    print(tf.shape(feature_extractor(image[0].numpy(), return_tensors='tf')['pixel_values']))
    print(feature_extractor(image[0].numpy()))
    break

In [ ]:
def process_example(example):
    print(example[0])
    inputs = feature_extractor(example[0].numpy(), return_tensors='tf')
    # print(inputs)
    inputs['labels'] = example[1]
    return inputs

In [ ]:
# temp_set = training_set.map(lambda x, y: (tf.experimental.numpy.moveaxis(x, -1, 1), y)) 
# valid_set = validation_set.map(lambda x, y: (tf.experimental.numpy.moveaxis(x, -1, 1), y))

# temp_set = temp_set.map(lambda x, y: process_example([x,y])) 
# valid_set = valid_set.map(lambda x, y: process_example([x,y])) 

# # results = []
# # for image,label in temp_set.take(1):
# #     results.append(process_example([image, label]))
# #     # print(tf.shape(data[1]))
# #     break

In [ ]:
# inputs
pixel_values = tf.keras.layers.Input(shape=(3,224,224), name='pixel_values', dtype='float32')

# model layer
vit = model.vit(pixel_values)[0]
classifier = tf.keras.layers.Dense(5, activation='softmax', name='outputs')(vit[:, 0, :])

# model
keras_model = tf.keras.Model(inputs=pixel_values, outputs=classifier)

In [ ]:
def preprocess_images(examples):

    print(examples.shape)
    images = examples['img']
    images = [np.array(image, dtype=np.uint8) for image in images]
    images = [np.moveaxis(image, source=-1, destination=0) for image in images]
    inputs = feature_extractor(images=images)
    examples['pixel_values'] = inputs['pixel_values']

    return examples.prefetch(buffer_size=AUTOTUNE)

In [ ]:
temp_set = training_set.map(lambda x, y: (tf.experimental.numpy.moveaxis(x, -1, 1), y)) 
valid_set = validation_set.map(lambda x, y: (tf.experimental.numpy.moveaxis(x, -1, 1), y)) 

In [ ]:
tf.keras.backend.clear_session()

start_time = time.time()

## Compiling the model
keras_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

vit_history = keras_model.fit(
  temp_set,
  validation_data=valid_set,
  epochs=EPOCHS
)

elasped_time = time.time() - start_time
model_elapsed_time["vit"] = elasped_time